In [1]:
import energyflow
import numpy as np
import tensorflow as tf
from tensorflow import keras

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
X, Y = energyflow.datasets.qg_jets.load(num_data=100000, generator='pythia', pad=True, with_bc=False, cache_dir='~/.energyflow')

In [3]:
# preprocess by centering jets and normalizing pts
for x in X:
    mask = x[:,0] > 0
    yphi_avg = np.average(x[mask,1:3], weights=x[mask,0], axis=0)
    x[mask,1:3] -= yphi_avg
    x[mask,0] /= x[:,0].sum()

In [4]:
# preprocess PIDs so they are O(1) or less
X[:,:,3] = X[:,:,3] / 2000

In [5]:
nparticles = X.shape[1]

# PyTorch

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.c1 = nn.Conv1d(4, 16, 1)
        self.p1 = nn.AvgPool1d(nparticles)
        self.fc1 = nn.Linear(16, 16)
        self.fc2 = nn.Linear(16, 1)

    def forward(self, x):
        x = F.relu(self.c1(x))
        x = self.p1(x)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

model = Net()
print(model)


Net(
  (c1): Conv1d(4, 16, kernel_size=(1,), stride=(1,))
  (p1): AvgPool1d(kernel_size=(139,), stride=(139,), padding=(0,))
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=1, bias=True)
)


In [31]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)

369


In [32]:
X_torch = torch.tensor(X, dtype=torch.float).transpose(1, 2)

In [33]:
# model(X_torch)

In [34]:
Y_torch = torch.tensor(Y, dtype=torch.float).unsqueeze(1)

In [35]:
from torch.utils.data import Dataset, DataLoader

class QGDataset(Dataset):
    """Quark/Gluon dataset from energyflow"""

    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
    
    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]


In [36]:
dataset = QGDataset(X_torch, Y_torch)
dataloader = DataLoader(dataset, batch_size=5,
                        shuffle=False)

In [37]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

In [38]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # geat the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.669
[1,  4000] loss: 0.563
[1,  6000] loss: 0.540
[1,  8000] loss: 0.522
[1, 10000] loss: 0.519
[1, 12000] loss: 0.509
[1, 14000] loss: 0.511
[1, 16000] loss: 0.502
[1, 18000] loss: 0.493
[1, 20000] loss: 0.499
[2,  2000] loss: 0.489
[2,  4000] loss: 0.488
[2,  6000] loss: 0.487
[2,  8000] loss: 0.478
[2, 10000] loss: 0.485
[2, 12000] loss: 0.481
[2, 14000] loss: 0.491
[2, 16000] loss: 0.482
[2, 18000] loss: 0.479
[2, 20000] loss: 0.488
[3,  2000] loss: 0.479
[3,  4000] loss: 0.481
[3,  6000] loss: 0.479
[3,  8000] loss: 0.472
[3, 10000] loss: 0.481
[3, 12000] loss: 0.477
[3, 14000] loss: 0.486
[3, 16000] loss: 0.478
[3, 18000] loss: 0.476
[3, 20000] loss: 0.484
[4,  2000] loss: 0.474
[4,  4000] loss: 0.478
[4,  6000] loss: 0.475
[4,  8000] loss: 0.468
[4, 10000] loss: 0.478
[4, 12000] loss: 0.474
[4, 14000] loss: 0.482
[4, 16000] loss: 0.474
[4, 18000] loss: 0.474
[4, 20000] loss: 0.482
[5,  2000] loss: 0.472
[5,  4000] loss: 0.475
[5,  6000] loss: 0.473
[5,  8000] 

In [65]:
# dataset_test = QGDataset(X_torch[:1000], Y_torch[:1000])
# dataloader_test = DataLoader(dataset_test, batch_size=5,
#                         shuffle=False)

In [63]:
def model_accuracy(model, data_x, data_y):
    # data_x and data_y are numpy matrices
    X = torch.Tensor(data_x)
    Y = torch.ByteTensor(data_y)   # a Tensor of 0s and 1s
    oupt = model(X)            # a Tensor of floats
    pred_y = oupt >= 0.5       # a Tensor of 0s and 1s
    num_correct = torch.sum(Y==pred_y)  # a Tensor
    acc = (num_correct.item() * 100.0 / len(data_y))  # scalar
    print('Accuracy: %d %%' % (acc))

In [64]:
model_accuracy(model, X_torch, Y_torch.byte())

Accuracy: 78 %
